<a href="https://colab.research.google.com/github/rmynzl/calismalarim/blob/main/234329040_R%C3%BCmeysa_Nazli_YSA_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Gerekli kütüphaneler ve veri yükleme

In [ ]:
import os
import tensorflow as tf
from google.colab import drive
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121, MobileNet
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
import seaborn as sns

# Mixed Precision kullanımı (GPU Performansı için)
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')



# Google Drive bağlıyorum
drive.mount('/content/drive')

#  Veri Seti Konumu
data_dir = '/content/drive/MyDrive/data'
if not os.path.exists(data_dir):
    raise FileNotFoundError(f"Hata: {data_dir} bulunamadı. Doğru yolu verdiğinden emin ol!")

# Görsel boyutu ve batch size belirliyorum
image_size = (224, 224)
batch_size = 64

# Veri artırımı ve ön işleme adımı
datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


* Google Drive'dan veri setini aldım, görseller için veri artırımı yaptım. Ayrıca, GPU performansını artırmak için mixed precision (float16) kullandım. Görsellerin boyutunu 224x224 olarak ayarladım ve batch size 64 olarak belirledim.  (eğitim kodu yavaş çalıştığı için batch size artırmak zorunda kaldım.)

* 0.20 test verisi olacak şekilde ayrım yapıyorum.


In [ ]:
# Eğitim ve doğrulama veri setlerini yükleme

train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode="sparse",
    subset="training",
    color_mode="rgb",
    shuffle=True
)

val_generator = datagen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode="sparse",
    subset="validation",
    color_mode="rgb",
    shuffle=False
)

Found 6043 images belonging to 2 classes.
Found 1510 images belonging to 2 classes.


* Veri setlerini yüklerken, train_generator ve val_generator kullanarak eğitim ve doğrulama verilerini ayırıyorum. Bu işlemde, datagen.flow_from_directory fonksiyonunu kullanıyorum. Eğitim ve doğrulama verisi, belirlediğim data_dir dizininden yükleniyor ve görüntüler image_size (224x224) boyutlarına getirilip, batch_size olarak 64 ile gruplanıyor.

* Eğitim verisi: 6043 adet görüntü bulunuyor ve bu görüntüler 2 farklı sınıfa ait.  (maskeli ve maskesiz grup)
* Doğrulama verisi: 1510 adet görüntü bulunuyor ve bunlar da 2 farklı sınıfa ait.  (maskeli ve maskesiz grup)

In [ ]:
from tensorflow.keras import mixed_precision
from tensorflow.keras.applications import DenseNet121, MobileNet
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Mixed Precision kullanımı (GPU Performansı için)
mixed_precision.set_global_policy('mixed_float16')

# Modeli oluşturma fonksiyonunu yazaluım
def create_model(base_model_name="MobileNet"):
    if base_model_name == "DenseNet121":
        base_model = DenseNet121(weights="imagenet", include_top=False, input_shape=(image_size[0], image_size[1], 3))
    elif base_model_name == "MobileNet":
        base_model = MobileNet(weights="imagenet", include_top=False, input_shape=(image_size[0], image_size[1], 3))
    else:
        raise ValueError("Geçersiz model adı. 'DenseNet121' veya 'MobileNet' olmalı.")

    # Önceden eğitilmiş katmanları donduruyorum daha hızlı olması için
    base_model.trainable = False

    # Modelin Üst Katmanlarını belirleyelim
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation="relu")(x)
    x = Dense(64, activation="relu")(x)
    predictions = Dense(2, activation="softmax")(x)  # Binary sınıflandırma için softmax

    model = Model(inputs=base_model.input, outputs=predictions)
    return model


* Mixed Precision: GPU hızlandırması için mixed_precision kullanarak işlem yapılan veri tipini float16 olarak ayarladım.

* Model oluşturma: DenseNet121 veya MobileNet bazlı pre-trained modelleri seçtim. Bu modellerin üst katmanlarını özelleştirdim.

* Eğitim için hazır model: Pre-trained modelin katmanlarını dondurdum, ardından modelin üst katmanlarına dropout ve dense katmanları ekledim. Sonuç olarak iki sınıflı (binary) sınıflandırma yapan bir model oluşturmuş oldum.

In [ ]:
# Modeli oluşturuyorum
optimizer = Adam(learning_rate=0.001)
model = create_model(base_model_name="MobileNet")
model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])


* Bu kodda modelimi oluşturup derliyorum.

* İlk olarak, Adam optimizasyon yöntemini seçiyorum ve öğrenme oranını(LR) 0.001 olarak ayarlıyorum.

* Sonra, create_model fonksiyonu ile MobileNet tabanlı modelimi oluşturuyorum.

* Son olarak, modelimi derlerken kayıp fonksiyonu olarak sparse_categorical_crossentropy ve başarı metriği olarak accuracy belirliyorum. Bu şekilde modelim eğitim için hazır hale gelmiş oluyor.

### Callback tanımlama ve Model Eğitimi

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# 📌 Callbacks tanımlayalım
early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True, verbose=1)
lr_reduction = ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=2, verbose=1)
checkpoint = ModelCheckpoint("best_model.h5", save_best_only=True, monitor="val_loss", verbose=1)

# 📌 Modeli Eğitelim
history = model.fit(
    train_generator,
    epochs=30,
    validation_data=val_generator,
    callbacks=[early_stopping, lr_reduction, checkpoint],
    verbose=1
)


Epoch 1/30
24/95 ━━━━━━━━━━━━━━━━━━━━ 28:12 24s/step - accuracy: 0.8177 - loss: 0.3371

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


95/95 ━━━━━━━━━━━━━━━━━━━━ 0s 25s/step - accuracy: 0.9181 - loss: 0.1682 
Epoch 1: val_loss improved from inf to 0.08923, saving model to best_model.h5


95/95 ━━━━━━━━━━━━━━━━━━━━ 3162s 33s/step - accuracy: 0.9185 - loss: 0.1674 - val_accuracy: 0.9722 - val_loss: 0.0892 - learning_rate: 0.0010
Epoch 2/30
95/95 ━━━━━━━━━━━━━━━━━━━━ 0s 13s/step - accuracy: 0.9852 - loss: 0.0442 
Epoch 2: val_loss improved from 0.08923 to 0.05915, saving model to best_model.h5


95/95 ━━━━━━━━━━━━━━━━━━━━ 1600s 16s/step - accuracy: 0.9852 - loss: 0.0441 - val_accuracy: 0.9861 - val_loss: 0.0592 - learning_rate: 0.0010
Epoch 3/30
95/95 ━━━━━━━━━━━━━━━━━━━━ 0s 13s/step - accuracy: 0.9854 - loss: 0.0463 
Epoch 3: val_loss improved from 0.05915 to 0.03129, saving model to best_model.h5


95/95 ━━━━━━━━━━━━━━━━━━━━ 1565s 16s/step - accuracy: 0.9854 - loss: 0.0462 - val_accuracy: 0.9914 - val_loss: 0.0313 - learning_rate: 0.0010
Epoch 4/30
95/95 ━━━━━━━━━━━━━━━━━━━━ 0s 13s/step - accuracy: 0.9874 - loss: 0.0315 
Epoch 4: val_loss did not improve from 0.03129
95/95 ━━━━━━━━━━━━━━━━━━━━ 1553s 16s/step - accuracy: 0.9874 - loss: 0.0315 - val_accuracy: 0.9907 - val_loss: 0.0341 - learning_rate: 0.0010
Epoch 5/30
95/95 ━━━━━━━━━━━━━━━━━━━━ 0s 13s/step - accuracy: 0.9865 - loss: 0.0408 
Epoch 5: val_loss improved from 0.03129 to 0.02746, saving model to best_model.h5


95/95 ━━━━━━━━━━━━━━━━━━━━ 1584s 17s/step - accuracy: 0.9865 - loss: 0.0407 - val_accuracy: 0.9881 - val_loss: 0.0275 - learning_rate: 0.0010
Epoch 6/30
95/95 ━━━━━━━━━━━━━━━━━━━━ 0s 13s/step - accuracy: 0.9943 - loss: 0.0210 
Epoch 6: val_loss did not improve from 0.02746
95/95 ━━━━━━━━━━━━━━━━━━━━ 1584s 17s/step - accuracy: 0.9942 - loss: 0.0210 - val_accuracy: 0.9887 - val_loss: 0.0307 - learning_rate: 0.0010
Epoch 7/30
95/95 ━━━━━━━━━━━━━━━━━━━━ 0s 13s/step - accuracy: 0.9916 - loss: 0.0259 
Epoch 7: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.

Epoch 7: val_loss did not improve from 0.02746
95/95 ━━━━━━━━━━━━━━━━━━━━ 1570s 17s/step - accuracy: 0.9916 - loss: 0.0260 - val_accuracy: 0.9894 - val_loss: 0.0403 - learning_rate: 0.0010
Epoch 8/30
95/95 ━━━━━━━━━━━━━━━━━━━━ 0s 13s/step - accuracy: 0.9930 - loss: 0.0176 

In [ ]:
# Eğitim ve doğrulama doğruluğu/grafiği çizdirelim

def plot_learning_curves(history):
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.legend()
    plt.title('Accuracy')

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.legend()
    plt.title('Loss')
    plt.show()

plot_learning_curves(history)

* Burada, eğitim sürecindeki doğruluk (accuracy) ve kayıp (loss) değerlerini görselleştiriyoruz.

* İlki, eğitim doğruluğunu (accuracy) gösteriyor. İkincisi ise eğitim kaybı (loss) ile doğrulama kaybını (val_loss) karşılaştırıyor.









### Değerlendirme Aşamaları

> Blok alıntı ekle



In [ ]:
# Test set ile değerlendirme yapalım
X_test, y_test = next(val_generator)
predictions = model.predict(X_test)
y_pred = np.argmax(predictions, axis=1)

In [ ]:
# Karışıklık Matrisi

conf_matrix = confusion_matrix(y_test, y_pred)
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# ROC Eğrisi

fpr, tpr, _ = roc_curve(y_test, predictions[:, 1])
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, label='AUC = {:.3f}'.format(roc_auc))
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
plt.title('ROC Curve')
plt.show()


# Performans Metrikleri

print(classification_report(y_test, y_pred))